In [11]:
import os

In [12]:
%pwd

'/Users/mohammedsohailmirza/Downloads/BMW Predictions E2E/BMW-Predictions-E2E'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/mohammedsohailmirza/Downloads/BMW Predictions E2E/BMW-Predictions-E2E'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    min_samples_split: int
    min_samples_leaf: int
    max_features: str
    random_state: int
    n_jobs: int
    target_column: str

In [14]:
from car_price.constants import *
from car_price.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            max_features=params.max_features,
            random_state=params.random_state,
            n_jobs=params.n_jobs,
            target_column=self.params.features.target
        )

        return model_trainer_config

In [16]:
import os
from car_price import logger
import pandas as pd

In [17]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def load_data(self):
        """Load train and test data"""
        logger.info(f"Loading training data from {self.config.train_data_path}")
        train_df = pd.read_csv(self.config.train_data_path)
        
        logger.info(f"Loading test data from {self.config.test_data_path}")
        test_df = pd.read_csv(self.config.test_data_path)
        
        logger.info(f"Train data shape: {train_df.shape}")
        logger.info(f"Test data shape: {test_df.shape}")
        
        return train_df, test_df


    def prepare_data(self, train_df, test_df):
        """Prepare features and target variables"""
        logger.info("Preparing features and target variables")
        
        # Separate features and target
        X_train = train_df.drop(columns=[self.config.target_column])
        y_train = train_df[self.config.target_column]
        
        X_test = test_df.drop(columns=[self.config.target_column])
        y_test = test_df[self.config.target_column]
        
        logger.info(f"Feature columns: {list(X_train.columns)}")
        logger.info(f"Number of features: {X_train.shape[1]}")
        
        return X_train, y_train, X_test, y_test


    def train_model(self, X_train, y_train):
        """Train Random Forest model with specified hyperparameters"""
        logger.info("="*60)
        logger.info("Training Random Forest Regressor")
        logger.info("="*60)
        
        # Log hyperparameters
        logger.info("Hyperparameters:")
        logger.info(f"  n_estimators: {self.config.n_estimators}")
        logger.info(f"  max_depth: {self.config.max_depth}")
        logger.info(f"  min_samples_split: {self.config.min_samples_split}")
        logger.info(f"  min_samples_leaf: {self.config.min_samples_leaf}")
        logger.info(f"  max_features: {self.config.max_features}")
        logger.info(f"  random_state: {self.config.random_state}")
        logger.info(f"  n_jobs: {self.config.n_jobs}")
        
        # Initialize model with hyperparameters from params.yaml
        model = RandomForestRegressor(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf,
            max_features=self.config.max_features,
            random_state=self.config.random_state,
            n_jobs=self.config.n_jobs
        )
        
        # Train the model
        logger.info("Starting model training...")
        import time
        start_time = time.time()
        
        model.fit(X_train, y_train)
        
        training_time = time.time() - start_time
        logger.info(f"Model training completed in {training_time:.2f} seconds")
        
        return model


    def save_model(self, model):
        """Save trained model"""
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(model, model_path)
        logger.info(f"Model saved to: {model_path}")


    def train(self):
        """Main training pipeline"""
        try:
            # Load data
            train_df, test_df = self.load_data()
            
            # Prepare data
            X_train, y_train, X_test, y_test = self.prepare_data(train_df, test_df)
            
            # Train model
            model = self.train_model(X_train, y_train)
            
            # Save model
            self.save_model(model)
            
            logger.info("Model training completed successfully")
            
        except Exception as e:
            logger.error(f"Error in model training: {e}")
            raise e

In [21]:
from sklearn.ensemble import RandomForestRegressor
import joblib

In [22]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2026-01-18 23:57:30,928: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-18 23:57:30,932: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-18 23:57:30,934: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-18 23:57:30,934: INFO: common: created directory at: artifacts]
[2026-01-18 23:57:30,935: INFO: common: created directory at: artifacts/model_trainer]
[2026-01-18 23:57:30,936: INFO: 1054161621: Loading training data from artifacts/data_transformation/train.csv]
[2026-01-18 23:57:30,957: INFO: 1054161621: Loading test data from artifacts/data_transformation/test.csv]
[2026-01-18 23:57:30,961: INFO: 1054161621: Train data shape: (8624, 10)]
[2026-01-18 23:57:30,962: INFO: 1054161621: Test data shape: (2157, 10)]
[2026-01-18 23:57:30,962: INFO: 1054161621: Preparing features and target variables]
[2026-01-18 23:57:30,963: INFO: 1054161621: Feature columns: ['year', 'mileage', 'tax', 'mpg', 'engineSize', 'car_age', 'model_enc